In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import pathlib

Load Dataset

In [ ]:
data_dir = pathlib.Path("/content/drive/MyDrive/Main Project/KernellQ/Raw Dataset")

batch_size = 32
img_height = 150
img_width = 150

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Preprocessing (normalisasi)
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))

Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

CNN Model

In [ ]:
from tensorflow.keras import Input

model = models.Sequential([
    Input(shape=(150, 150, 3)),   # <--- wajib untuk build model
    data_augmentation,
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(512, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

Callback

In [ ]:
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

target_acc = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    mode='max',
    baseline=0.99,   
    restore_best_weights=True
)

callbacks_list = [early_stop, target_acc]

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',   # <--- ganti ini
    metrics=['accuracy']
)

Training

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks_list
)

Visualisasi Training vs Validation Accuracy

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8,4))
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Training & Validation Accuracy')
plt.show()

plt.figure(figsize=(8,4))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.title('Training & Validation Loss')
plt.show()

Simpan Model

In [ ]:
model.save('/content/drive/MyDrive/Main Project/KernellQ/kernellq_cnn.h5')

Prediksi Gambar Baru

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from google.colab import files

train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/Main Project/KernellQ/Raw Dataset",
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds_raw.class_names  
print("Kelas yang dikenali model:", class_names)

uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename
    print("File yang dipilih:", img_path)

    img = image.load_img(img_path, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) / 255.0

    pred = model.predict(x)
    predicted_class = class_names[np.argmax(pred)]
    confidence = round(np.max(pred), 4)

    print("Prediksi:", predicted_class, "| Probabilitas:", confidence)
